In [56]:
from pyspark import SparkContext, SparkConf

In [57]:
from operator import add

In [58]:
conf = SparkConf().setMaster("local[*]")
sc = SparkContext.getOrCreate()

In [59]:
def rdd_to_dictionary(rdd):
    return sc.parallelize(rdd).collectAsMap()

In [60]:
def write_json(dictionary,filename):
    import json  

    with open(filename, "w") as outfile:  
        json.dump(dictionary, outfile)    

In [71]:
def trimPunctuation(word):
    if word[0] in punctuation:
        return word[1:]
    elif word[-1] in punctuation:
        return word[:-1]
    else:
        return word

In [72]:
documents = ["data/4300-0.txt","data/pg19033.txt", "data/pg36.txt","data/pg514.txt","data/pg1497.txt","data/pg3207.txt","data/pg42671.txt","data/pg6130.txt"]
data_bag = spark.read.text(documents).rdd.map(lambda r: r[0])

In [73]:
words_count = data_bag.flatMap(lambda x: x.split(' '))\
                .map(lambda x: x.lower()) \
                .map(lambda x: (x, 1))\
                .reduceByKey(add)\
                .filter(lambda x: x[1] > 1)\
                .filter(lambda x: len(x[0]) > 0)
#print(words_count.count())

In [74]:
punctuation = [".",",",":",";","'","!","?"];
words_without_punctuation = words_count.map(lambda x: (trimPunctuation(x[0]), x[1])).reduceByKey(add)
#print(words_without_punctuation.count())

In [75]:
stop_words_data_bag = spark.read.text("data/stopwords.txt").rdd.map(lambda r: r[0])
stop_words = stop_words_data_bag.flatMap(lambda x: x.split('\n'))\
                .map(lambda x: x.lower()).map(lambda x: (x, 1))\
                .reduceByKey(add)\
                .filter(lambda x: len(x[0]) > 0)
# print(stop_words.count())

In [76]:
words_without_punct_stop_words = words_without_punctuation.subtractByKey(stop_words)

# print(words_without_punct_stop_words.count())

In [77]:
top_40_words_without_punct_stop_words = words_without_punct_stop_words.takeOrdered(40, key = lambda x: -x[1])


In [78]:
top_40_words_without_punct_stop_words_dictionary = rdd_to_dictionary(top_40_words_without_punct_stop_words)
# print(top_40_words_without_punct_stop_words_dictionary)

In [79]:
write_json(top_40_words_without_punct_stop_words_dictionary,"sp3.json")

In [80]:
sc.stop()